In [10]:
import torch
from PIL import Image

from lavis.models import load_model_and_preprocess

/home/disi/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/disi/miniconda3/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/disi/miniconda3/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
/home/disi/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated

In [27]:
dataset_root = 'datasets/ADE20K_2021_17_01/images/ADE/validation/cultural/apse__indoor/'
raw_image = Image.open(f"{dataset_root}ADE_val_00000025.jpg").convert('RGB')


In [11]:
device = 'cuda'
# we associate a model with its preprocessors to make it easier for inference.
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)
# uncomment to use base model
# model, vis_processors, _ = load_model_and_preprocess(
#     name="blip_caption", model_type="base_coco", is_eval=True, device=device
# )
vis_processors.keys()

/home/disi/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/disi/miniconda3/lib/python3.10/site-packages/lavis/models/base_model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

dict_keys(['train', 'eval'])

In [29]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [30]:
image.shape

torch.Size([1, 3, 384, 384])

In [52]:
captions = model.generate({"image": image}, use_nucleus_sampling=True, num_captions=10)
type(captions)

list

In [4]:
import os
import glob
from PIL import Image
from torch.utils.data import DataLoader, Dataset

class ADE20KDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        
        with open('datasets/ADE20K_2021_17_01/image_paths.txt') as f:
            self.image_paths = [x.strip() for x in f.read().splitlines()]
        
        print("Loaded dataset with {} images".format(len(self.image_paths)))
       

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

In [12]:
from torchvision import transforms
dataset_root = '/datasets/ADE20K_2021_17_01/images'
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])
data = ADE20KDataset(dataset_root, transform=transform)

Loaded dataset with 27574 images


In [13]:
data.__getitem__(0)

tensor([[[0.0588, 0.0549, 0.0471,  ..., 0.0902, 0.1059, 0.0510],
         [0.8235, 0.8196, 0.8157,  ..., 0.7333, 0.6784, 0.2667],
         [0.8471, 0.8431, 0.8471,  ..., 0.8471, 0.8157, 0.2667],
         ...,
         [0.0627, 0.0667, 0.1020,  ..., 0.3804, 0.2588, 0.0745],
         [0.1059, 0.0902, 0.0980,  ..., 0.3843, 0.2549, 0.0706],
         [0.1098, 0.0902, 0.0980,  ..., 0.3882, 0.2549, 0.0667]],

        [[0.0157, 0.0157, 0.0157,  ..., 0.0353, 0.0588, 0.0157],
         [0.7765, 0.7765, 0.7765,  ..., 0.6667, 0.6235, 0.2235],
         [0.7961, 0.7961, 0.7961,  ..., 0.7569, 0.7373, 0.2000],
         ...,
         [0.0431, 0.0431, 0.0667,  ..., 0.4000, 0.2667, 0.0745],
         [0.0863, 0.0667, 0.0627,  ..., 0.4078, 0.2667, 0.0706],
         [0.0902, 0.0667, 0.0627,  ..., 0.4118, 0.2627, 0.0667]],

        [[0.0549, 0.0510, 0.0431,  ..., 0.0118, 0.0118, 0.0039],
         [0.8157, 0.8157, 0.8118,  ..., 0.5882, 0.5451, 0.1490],
         [0.8353, 0.8353, 0.8392,  ..., 0.6784, 0.6667, 0.

In [14]:
dataloader = DataLoader(data, batch_size=16, shuffle=True)

In [ ]:
from tqdm import tqdm
captions = []
for batch in tqdm(dataloader, desc="Generating captions", unit="batch"):
    captions.extend(model.generate({"image": batch.to(device)}, use_nucleus_sampling=True, num_captions=10))

vocabulary = set([cap.split(" ") for cap in captions])
print(len(vocabulary))

Generating captions:   0%|          | 0/1724 [00:00<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 110.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 64.19 MiB is free. Including non-PyTorch memory, this process has 15.70 GiB memory in use. Of the allocated memory 15.20 GiB is allocated by PyTorch, and 133.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 